# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
!pip install chromadb ollama langchain_huggingface sentence-transformers google-generativeai --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew,LLM

In [4]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"


groq_llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

In [5]:
llmconfig = dict(
    llm=dict(
        provider="openai",
        config=dict(
            model="ibm-granite/granite-3.3-2b-instruct",
            base_url=f"{INFERENCE_SERVER_URL}/v1",
            api_key=API_KEY,
        ),
    ),
    embedder=dict(
        provider="huggingface",#,.goolge
        config=dict(
            model="BAAI/bge-small-en-v1.5"#"models/embedding-001"#"nomic-ai/nomic-embed-text-v1"
        ),
    ),
)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [7]:
# Define file paths for YAML configurations
files = {
    'agents': 'L5-config/agents.yaml',
    'tasks': 'L5-config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

website_search_tool = WebsiteSearchTool(
config=dict(
        llm=dict(
            provider="openai", # Options include ollama, google, anthropic, llama2, and more
            config=dict(
                model="ibm-granite/granite-3.3-2b-instruct",
                base_url=f"{INFERENCE_SERVER_URL}/v1",
                api_key=API_KEY,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
            ),
        ),
    )
)

/opt/app-root/lib64/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/opt/app-root/lib64/python3.11/site-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/opt/app-root/lib64/python3.11/site-packages/embedchain/embedder/huggingface.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `

## Setup Multi LLM models

In [9]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
# groq_llm = "groq/llama-3.1-70b-versatile"
os.environ['SERPER_API_KEY'] ="30bdec95f601d48ca27880feedef448ca840b2f0"

## Creating Crew, Agents, and Tasks

In [10]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), website_search_tool],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), website_search_tool],
    llm=groq_llm,
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=groq_llm,
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [11]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4f9801c3-ad0f-4d2c-aacb-4d5a2a79a3c1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Market Analyst                                                                                     │
│                                                                                                                 │
│  Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular      │
│  focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most       │
│  relevant and impactful news items that could influence market trends or investor decisions. Utilize financial  │
│  news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends,         │
│  regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the     │
│  impact on the stock market in 2024.                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Market Analyst                                                                                     │
│                                                                                                                 │
│  Thought: Thought: I should utilize the Search the internet with Serper tool to find the latest articles and    │
│  news related to inflation in the US and its impact on the stock market in 2024. I will also need to access     │
│  specific financial news APIs for real-time market data.                                                        │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Thought: I now have a comprehensive report and social media updates ready to craft detailed content pieces in  │
│  a well-structured markdown format. Below, I've compiled a detailed and visually appealing article along with   │
│  relevant social media posts formatted according to the provided guidelines:                                    │
│                                                                                                                 │
│  ### Article:                                                                                                   │
│                                                                                                                 │
│  # Navigating Inflation's Complex Dance: Impacts on the Stock Market in 2024                                    │
│                                                                                                                 │
│  ## **Introduction**                                                                                            │
│                                                                                                                 │
│  In 2024, inflation dynamics in the United States created a convoluted interplay influencing the stock market   │
│  and investor behavior. This article examines the intricate relationship between rising cost structures and     │
│  their effects on the thriving stock market, drawing upon leading economic forecasts, market analyses, and      │
│  expert opinions.                                                                                               │
│                                                                                                                 │
│  ## **1. Deloitte’s Economic Reasoning (Q2 2025)**                                                              │
│                                                                                                                 │
│  According to Deloitte's U.S. Economic Forecast for Q2 2025, inflation is projected to accelerate due to        │
│  tariffs. This acceleration implies a more extensive cost structure for businesses, potentially leading to      │
│  increased profit margins if market demand perseveres. However, watch for market volatility if the Federal      │
│  Reserve responds with more aggressive rate increases to combat inflation.                                      │
│                                                                                                                 │
│  **Visualization:** Hypothetical chart juxtaposing inflation trends and stock market indices over the           │
│  specified period under divergent monetary policy scenarios.                                                    │
│                                                                                                                 │
│  ## **2. Trading Economics' Inflation Outlook**                                                                 │
│                                                                                                                 │
│  Trading Economics projects the U.S. Consumer Price Index (CPI) to average 2.40% from December 2024 through     │
│  2025, signaling a return to more standard inflation rates. If the Federal Reserve aligns with this             │
│  moderation, a low-rate environment persisting could be

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e057b4e3-05ba-4a11-92ab-7de9b0133c17                                                                     │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4f9801c3-ad0f-4d2c-aacb-4d5a2a79a3c1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Thought: I now have a comprehensive report and social media updates ready to craft detailed      │
│  content pieces in a well-structured markdown format. Below, I've compiled a detailed and visually appealing    │
│  article along with relevant social media posts formatted according to the provided guidelines:                 │
│                                                                                                                 │
│  ### Article:                                                                                                   │
│                                                                                                                 │
│  # Navigating Inflation's Complex Dance: Impacts on the Stock Market in 2024                                    │
│                                                                                                                 │
│  ## **Introduction**                                                                                            │
│                                                                                                                 │
│  In 2024, inflation dynamics in the United States created a convoluted interplay influencing the stock market   │
│  and investor behavior. This article examines the intricate relationship between rising cost structures and     │
│  their effects on the thriving stock market, drawing upon leading economic forecasts, market analyses, and      │
│  expert opinions.                                                                                               │
│                                                                                                                 │
│  ## **1. Deloitte’s Economic Reasoning (Q2 2025)**                                                              │
│                                                                                                                 │
│  According to Deloitte's U.S. Economic Forecast for Q2 2025, inflation is projected to accelerate due to        │
│  tariffs. This acceleration implies a more extensive cost structure for businesses, potentially leading to      │
│  increased profit margins if market demand perseveres. However, watch for market volatility if the Federal      │
│  Reserve responds with more aggressive rate increases to combat inflation.                                      │
│                                                                                                                 │
│  **Visualization:** Hypothetical chart juxtaposing inflation trends and stock market indices over the           │
│  specified period under divergent monetary policy scenarios.                                                    │
│                                                                                                                 │
│  ## **2. Trading Economics' Inflation Outlook**                                                                 │
│                                                                                                                 │
│  Trading Economics projects the U.S. Consumer Price Index (CPI) to average 2.40% from December 2024 through     │
│  2025, signaling a return to more standard inflation r

## Social Content

In [12]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Social Media Platform
🚀**News Alert: Delve into our comprehensive
analysis of Deloitte, Trading Economics, Nasdaq,
Hartford Funds, plus expert insights on platforms
like MarketWatch, Financial Times, and Bloomberg.
How is inflation shaping the stock market in 2024?
[Link to articles]**
--------------------------------------------------
Social Media Platform
💡**Inflation projections and their influence on
the stock market in 2024. Gain critical market
insights, drawing from leading economic studies
and market trend-setting platforms.
[Growth](https://www.marketwatch.com/) |
[Business](https://www.financialtimes.com/) |
[Wall Street Journal](https://www.bloomberg.com/)
--------------------------------------------------


/tmp/ipykernel_3613/3812343593.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  posts = result.pydantic.dict()['social_media_posts']


## Blog Post

In [13]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

/tmp/ipykernel_3613/3784749904.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  display(Markdown(result.pydantic.dict()['article']))


Navigating Inflation's Complex Dance: Impacts on the Stock Market in 2024
## Introduction
In 2024, inflation dynamics in the United States created a convoluted interplay influencing the stock market and investor behavior. This article examines the intricate relationship between rising cost structures and their effects on the thriving stock market, drawing upon leading economic forecasts, market analyses, and expert opinions.
## 1. Deloitte’s Economic Reasoning (Q2 2025)
According to Deloitte's U.S. Economic Forecast for Q2 2025, inflation is projected to accelerate due to tariffs. This acceleration implies a more extensive cost structure for businesses, potentially leading to increased profit margins if market demand perseveres. However, watch for market volatility if the Federal Reserve responds with more aggressive rate increases to combat inflation.
**Visualization:** Hypothetical chart juxtaposing inflation trends and stock market indices over the specified period under divergent monetary policy scenarios.
## 2. Trading Economics' Inflation Outlook
Trading Economics projects the U.S. Consumer Price Index (CPI) to average 2.40% from December 2024 through 2025, signaling a return to more standard inflation rates. If the Federal Reserve aligns with this moderation, a low-rate environment persisting could benefit sectors like utilities and consumer staples.
**Data Visualization:** Graph showing the correlation between anticipated CPI rates and the hypothetical performance of key stock market sectors in 2024-2025.
## 3. Nasdaq's Stock Market Highs (December 2024)
The Nasdaq witnessed unprecedented back-to-back annual gains of 20% or more in December 2024, marking the first such anniversary since 1998. This anomaly is attributed to bullish investor sentiment potentially fueled by expectations of the Federal Reserve maintaining an accommodative stance.
**Infographic:** Visualization comparing Nasdaq climbing to historical highs, alongside key economic indicators and sentiment metrics.
## 4. Equities vs. Inflation (Hartford Funds)
Research from Hartford Funds indicates that equities commonly outperform inflation over periods with low to moderate inflation—suggesting ongoing resilience for equities under controlled inflationary environments.
**Thought-Provoking Question:**
Is the historical pattern of stock outperformance versus inflation in the face of moderate inflation a reliable indicator for the trajectory of 2024’s market?
## 5. Market Sentiment and Inflation Data (CNBC)
Positive CPI data in December 2024 bolstered bullish investor sentiment, steering market bulls toward anticipations of further Federal Reserve rate reductions. This scenario aligns with interpretations of the Fed’s intent to continue easing monetary policy against elevated inflation.
**Potential Implications:**
- Continued monetary expansion could amplify growth-oriented asset valuations.
- Sectors adept at leveraging economic cycles—such as industrials and technology—might navigate uncertain waters with relative efficiency despite mixed economic signals.
